In [60]:
import os

In [1]:
import typing
from typing import List, Callable
def strings_(n: int, d: int):
    ''' generates non-equivalent strings of length n using exactly d letters
    two strings on aphabet of the same size are equivalent if there is a bijection 
    between alphabets that makes strings equal
    using alphabet 1,2,3...
    
    valid input n >= 0, d >= 0
    This is combinatorial problem of paritioning a set into d classes; counted by Bell numbers
    '''
    if n < 0 or d < 0:
        raise ValueError
    if n == 0:
        if d == 0:
            yield []  # empty string
        # else nothing 
    else:
        if d > 0:
            for prefix in strings_(n-1, d-1):
                yield prefix + [d]   # append new letter to each element of the previous set
            for prefix in strings_(n-1, d):
                for letter in range(1, d+1):  # repeat one of existing letters
                    yield prefix + [letter]
        # else nothing

def strings(n):
    ''' generates all non-equivalent strings using d = 1...n different letters'''
    for d in range(1, n+1):
        yield from strings_(n, d)

def btree(n: int, offset: int):
    ''' generates binary trees with n leaves in postfix rep; 
    the counting is Catalan number C(n-1); number of correct 
    parenthesis expressions with n-1 open parenthesis; Dyck words; https://en.wikipedia.org/wiki/Dyck_language
    There are 66 interpretatinos in Stanley book on combinatorics 
    https://en.wikipedia.org/wiki/Catalan_number

    the arguments are denoted by their slot position (NOTICE: the semantics of 0 is different from postfix we used
    in serlib parser where 0 denoted the n-arity of the arguments; however this is temporary difference
    as we'll compose btree with slot substitutions)
    offset denotes the offset of argument labelling
    
    the sentinel -2 denotes application of a binary operator on the postfix
    
    valid input n: int, n >= 1
    '''
    if n < 1: 
        raise ValueError
    if n == 1:
        yield [offset]
    for k in range(1, n):
        for left in btree(k, offset):
            for right in btree(n-k, offset+k):
                yield left + right + [-2]
        

        
        
def exprs_(n, d):
    '''
    generates binary expressions with n leaves on d letters 
    direct product of Catalan generator with Bell generator
    '''
    for stack in btree(n, 0):
        for arg in strings_(n, d):
            yield [x if x < 0 else arg[x] for x in stack]
def exprs(n):
    for d in range(0, n+1):
        yield from exprs_(n, d)

        
def evaluate(e, op, arg: List[bool]):
    '''
    Boolean stack evaluator
    e: is binary tree expression in stack form
    arg: is the list of boolean values
    op: (bool, bool) -> bool binary operator in the tree nodes
    '''
    stack = []
    pos = 0
    while pos < len(e):
        c = e[pos]; pos += 1  # read element from expression postfix
        if c > 0 and c <= len(arg): # normal argument, place on stack
            stack.append(arg[c-1])
        elif c == -2:    # binary operator
            if len(stack) >= 2:
                a = stack.pop()
                b = stack.pop()
                res = op(b, a)
                stack.append(res)
            else:
                raise ValueError("bad postfix: not enough args for binary operator")
        else:
            raise ValueError("bad postfix: unknown element code")
    if len(stack) == 1:
        return stack.pop()
    else:
        raise ValueError("bad postfix, too many args for binary operators")

        
def sexp_op(a, b):
    #arrow = "\u2192"
    arrow = "->"
    return f"({a}{arrow}{b})"

def imply(a: bool, b: bool):
    return (not a) or b

def sexp(e): 
    return evaluate(e, sexp_op, [chr(ord('A')+i) for i in range(max(e))])

def product(values, offset):
    ''' generates from cartesian products of iterables passed as List
    offset is for recursion efficiency, to avoid list constructions
   
    (if LinkedList implementation, use head :: tail in recursion) 
    '''
    if offset > len(values):
        raise ValueError

    if (offset == len(values)):
        yield []
    else:
        for x in values[offset]:
            for rest in product(values, offset+1):
                yield [x] + rest

def valid(e):
    ''' brute force proving theorem
    in propositional calculus entails to check that expression evaluates to True for
    all input arguments 
    '''
    n_args = max(e)
    return all(evaluate(e, imply, arg) for arg in product([[False,True] for _ in range(n_args)], 0))  



def show_theorem_table(n):
    for e in exprs(n):
        print(sexp(e), valid(e))
def valid_theorems(n):
    ''' generates valid theorems by brute force on theorem expression and proof '''
    for e in exprs(n):
        if valid(e):
            yield e

In [2]:
list(len(list(strings(n))) for n in range(1, 12))   # famous https://en.wikipedia.org/wiki/Bell_number 

[1, 2, 5, 15, 52, 203, 877, 4140, 21147, 115975, 678570]

In [3]:
list(strings(3))

[[1, 1, 1], [1, 1, 2], [1, 2, 1], [1, 2, 2], [1, 2, 3]]

In [4]:
list(len(list(btree(n, 0))) for n in range(1, 12))   # famous https://en.wikipedia.org/wiki/Catalan_number

[1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862, 16796]

In [5]:
# Total numbero of theorems up to level n

In [6]:

from itertools import accumulate
list(enumerate([0]+list(accumulate(list(i*j for i,j in zip([1, 2, 5, 15, 52, 203, 877,4140, 21147, 115975, 678570]
                                        ,[1, 1, 2, 5, 14, 42, 132,429, 1430, 4862, 16796]))))))


[(0, 0),
 (1, 1),
 (2, 3),
 (3, 13),
 (4, 88),
 (5, 816),
 (6, 9342),
 (7, 125106),
 (8, 1901166),
 (9, 32141376),
 (10, 596011826),
 (11, 11993273546)]

In [7]:
list(btree(5,0))

[[0, 1, 2, 3, 4, -2, -2, -2, -2],
 [0, 1, 2, 3, -2, 4, -2, -2, -2],
 [0, 1, 2, -2, 3, 4, -2, -2, -2],
 [0, 1, 2, 3, -2, -2, 4, -2, -2],
 [0, 1, 2, -2, 3, -2, 4, -2, -2],
 [0, 1, -2, 2, 3, 4, -2, -2, -2],
 [0, 1, -2, 2, 3, -2, 4, -2, -2],
 [0, 1, 2, -2, -2, 3, 4, -2, -2],
 [0, 1, -2, 2, -2, 3, 4, -2, -2],
 [0, 1, 2, 3, -2, -2, -2, 4, -2],
 [0, 1, 2, -2, 3, -2, -2, 4, -2],
 [0, 1, -2, 2, 3, -2, -2, 4, -2],
 [0, 1, 2, -2, -2, 3, -2, 4, -2],
 [0, 1, -2, 2, -2, 3, -2, 4, -2]]

In [8]:
list(exprs_(3,3)) 

[[1, 2, 3, -2, -2], [1, 2, -2, 3, -2]]

In [9]:
list(exprs(3)) # (Catalan = 2)  x  (Bell = 5) = 10 conjectural theorems of length 3

[[1, 1, 1, -2, -2],
 [1, 1, -2, 1, -2],
 [1, 1, 2, -2, -2],
 [1, 2, 1, -2, -2],
 [1, 2, 2, -2, -2],
 [1, 1, -2, 2, -2],
 [1, 2, -2, 1, -2],
 [1, 2, -2, 2, -2],
 [1, 2, 3, -2, -2],
 [1, 2, -2, 3, -2]]

In [10]:
# S-expressions using the concatenation binary operator
for e in exprs_(3, 2):
    print(sexp(e))

(A->(A->B))
(A->(B->A))
(A->(B->B))
((A->A)->B)
((A->B)->A)
((A->B)->B)


In [11]:
l6 = list(exprs(6))

In [12]:
sexp(l6[1000])

'((A->((B->B)->(B->A)))->B)'

In [13]:
stat = []
for n in range(1, 9):
    stat.append( (n, len(list(exprs(n))),  len(list(valid_theorems(n)))))

In [14]:
stat

[(1, 1, 0),
 (2, 2, 1),
 (3, 10, 3),
 (4, 75, 25),
 (5, 728, 206),
 (6, 8526, 2298),
 (7, 115764, 28504),
 (8, 1776060, 409543)]

In [15]:
[x[1] for x in stat]          # the number of all binary expressions of length n https://oeis.org/A289679
# referred to paper https://arxiv.org/abs/1608.03912
# seems that paper is reproducing this notebook up to n = 14 :)

[1, 2, 10, 75, 728, 8526, 115764, 1776060]

In [16]:
[x[2] for x in stat]
# https://arxiv.org/pdf/1910.01775.pdf
# quoting the paper: all provable implicational intuitionistic propositional calculus
# formulas = NEW: 0, 1, 3, 24, 201, 2201, 27406, 391379, 6215192

[0, 1, 3, 25, 206, 2298, 28504, 409543]

google integer sequences gives Paul Tarau papers
- https://arxiv.org/abs/1608.03912   

- https://arxiv.org/pdf/1910.01775.pdf


notice that with brute force search we have slightly more valid formulas: Does it mean that there exists valid formulas 
of classical logic (provable by substitution table / Tarski / denotation) that are not provable in intutionistic logic (Heyting / inference rules)? Or we have a mistake? 
- us: [0, 1, 3, 25, 206, 2298, 28504, 409543]    
- PT: [0, 1, 3, 24, 201, 2201, 27406, 391379, 6215192] 

What is asymptotics of this sequence???


No, the discrepancy is not a bug, but a feature! There are valid formulas in classical logic / denotation / Tarski which are not provable in intuionistic logic (Heyting / inference rules). The first example is indeed on 4 variables, Peirce's formula (has index 31 in our table of binary trees with 4 leaves on 2 variables:

 ((a->b)->a)->a



In [17]:
peirce = list(exprs_(4, 2))[31]

In [18]:
sexp(peirce), valid(peirce)

('(((A->B)->A)->A)', True)

https://en.wikipedia.org/wiki/Peirce%27s_law

The Peirce's law can be used to substitute the law of excluded middle. If Peirce law is added as axiom to the minimal intutionistic propositional calculus, the theory becomes equivalent to the classical logic


Also the discrepancy of Peirce law turns out to be fundamental in computation: 
https://en.wikipedia.org/wiki/Call-with-current-continuation

# Export dataset

In [19]:
import random
import numpy as np

In [20]:
def make_dataset(n: int):
    valid_theorems = []
    invalid_theorems = []
    for e in exprs(n):
        if valid(e):
            valid_theorems.append(e)
        elif max(e) < n:
            invalid_theorems.append(e)
    random.shuffle(valid_theorems)
    random.shuffle(invalid_theorems)
    return valid_theorems, invalid_theorems

In [21]:
dataset_true, dataset_false = map(np.array, make_dataset(8))
len(dataset_true), len(dataset_false)

(409543, 1366088)

In [22]:
np.random.shuffle(dataset_false)
dataset_true.tofile('dataset_true.ndarray')
dataset_false[:409543].tofile('dataset_false.ndarray')

# Export dataset to database MongoDB as coq theorems

To install MongoDB on Ubuntu 18.04 follow precisely the steps 1234 at https://docs.mongodb.com/manual/tutorial/install-mongodb-on-ubuntu/#import-the-public-key-used-by-the-package-management-system  
Then execute steps 12345 to lauch MongoDB at https://docs.mongodb.com/manual/tutorial/install-mongodb-on-ubuntu/#run-mongodb-community-edition

In [26]:
from typing import Union,  NewType, Any, Dict, List, Callable, Tuple
import tqdm
import concurrent.futures 
import pymongo
from pymongo import MongoClient

IntPostFix = List[int]

DbKeys = Union[NewType('vars',str), 
               NewType('stmt',str), 
               NewType('proofs',str), 
               NewType('n_vars',str), 
               NewType('n_args',str),
              NewType('lex_pos', str)]
DbValues = Any

def latin_single(x: int) -> str:
    if x in range(26):
        return chr(ord('A') + x)
    else:
        raise ValueError("the value of input for latin_single must be in range(26)")

def variables(e: IntPostFix, encoding_map: Callable[[int], str]):
    return sorted(list(encoding_map(x-1) for x in set(e) if x > 0))


def db_create_entry(e: IntPostFix, n_args: int, n_vars: int, lex_pos: int) -> Dict[DbKeys, DbValues]:
    n_args = (len(e) + 1) // 2
    thm_proofs = {}
    entry = {'n_args':n_args,
            'n_vars':n_vars,
            'lex_pos':lex_pos,
            'postfix': e,
            'bool_denote': None}
    return entry
    
def db_init(n_range: range, db: pymongo.collection.Collection):
    ''' erases existing database with n_args in python range n_range
        populates with exhaustive list of all formulas
        in the implication segment of propositional calculus
    '''
    
    for n_args in tqdm.tqdm(n_range):
        db.delete_many({"n_args": n_args})
        data = []
        for n_vars in range(1, n_args+1):
            for lex_pos, e in enumerate(exprs_(n_args,n_vars)):
                data.append(db_create_entry(e, n_args, n_vars, lex_pos))
        db.insert_many(data)
    return db
                

def valid_helper(obj_id: int, e: IntPostFix) -> Tuple[int, bool]:
    return (obj_id, valid(e))


def db_update_all_proofs(db: pymongo.collection.Collection, proof_type: str, proof_func: Callable[[str],bool], max_workers=1):
    data = []
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(valid_helper, entry['_id'], entry['postfix']) for entry in db.find()]
        for future in tqdm.tqdm(concurrent.futures.as_completed(futures), 
                                total = len(futures), 
                               miniters=1):
            obj_id, result = future.result()
            data.append((obj_id, result))
    
    for obj_id, result in tqdm.tqdm(data, total=len(data), miniters=1):
        db.update_one({"_id": obj_id}, 
                     {
                         "$set": { "bool_denote": result}
                     }
                    )

def test_proof_func(str):
    return len(str) + 1000

In [27]:
#db = db_init(range(1,8), MongoClient().formulas.propositional.implication)  
#db_update_all_proofs(db, 'bool_denote', valid, max_workers=40)
#db.create_index([('n_args', pymongo.ASCENDING)], unique=False)
# MongoClient().close()

# uncomment and evaluate the above lines to initialize and populate persistent MongoDB database when run notebook for the first time
db = MongoClient().formulas.propositional.implication


In [29]:
len(list(db.find({'n_args':7})))

115764

## Coq Evaluation

In [30]:
from pycoq.agent import evaluate_agent, evaluate_agent_in_session
from pycoq.agent import auto_agent as agent

import pycoq.serapi

from typing import List
import asyncio

import pymongo
db =  pymongo.MongoClient().formulas.propositional.implication

In [31]:
def coq_proposition(postfix, n_args, n_vars, lex_pos) -> str:
    theorem_name = f"th_{n_args}_{n_vars}_{lex_pos}"
    prop_formula = sexp(postfix)
    prop_variables = " ".join([chr(ord('A')+i) for i in range(n_vars)])
    return f"Theorem {theorem_name}: forall {prop_variables}: Prop, {prop_formula}."

# Test of evaluate_agent function

- returns n >= 0 if the tactics auto n successfully removed all goals
- returns -1 if tactics auto n for all n < auto_limit failed
- returns -2 if the proposition to prove could not be parsed by coq engine

** should see 'test passed' in all cells below


In [75]:
coq_ctxt = pycoq.common.CoqContext(pwd=os.getcwd(), executable='', target='serapi_shell')
cfg = pycoq.opam.opam_serapi_cfg(coq_ctxt)

In [76]:

pycoq.agent.DEBUG = False  # switch True/False to see the debub messages

res = await evaluate_agent(cfg, agent,
                     "Theorem th_4_2_9: forall A B C D: Prop, A->(A->B)->(B->C)->(C->D)->D.", 
                     agent_parameters={'auto_limit':3})
print(res) 
print('test passed ' if res == -1 else 'failed')

process with 16498 started as
cmd:  ['opam', 'exec', '--root', '/home/pestun/dataset100', '--switch', 'ocaml-variants.4.07.1+flambda_coq-serapi.8.11.0+0.11.1', '--', 'sertop', '--topfile', 'serapi_shell']
cwd  /home/pestun/code/pycoq/notes
-1
test passed 


In [69]:
res = await evaluate_agent(cfg, agent,
                     "Theorem th_4_2_9: forall A B C D: Prop, A->(A->B)->(B->C)->(C->D)->D.", 
                     agent_parameters={'auto_limit':10})

print(res) 
print('test passed ' if res == 4 else 'failed')

process with 13557 started as
cmd:  ['opam', 'exec', '--root', '/home/pestun/dataset100', '--switch', 'ocaml-variants.4.07.1+flambda_coq-serapi.8.11.0+0.11.1', '--', 'sertop', '--topfile', 'serapi_shell']
cwd  /home/pestun/code/pycoq/notes
4
test passed 


In [70]:
res = await evaluate_agent(cfg, agent, 
                     "Theorem th_4_2_9: forall A B C D: Prop, A->(A->B)->(B->C)->(C->D)->F.", 
                     agent_parameters={'auto_limit':10})

print(res) 
print('test passed ' if res == -2 else 'failed')

process with 13565 started as
cmd:  ['opam', 'exec', '--root', '/home/pestun/dataset100', '--switch', 'ocaml-variants.4.07.1+flambda_coq-serapi.8.11.0+0.11.1', '--', 'sertop', '--topfile', 'serapi_shell']
cwd  /home/pestun/code/pycoq/notes
-2
test passed 


In [54]:
res = await evaluate_agent(cfg, agent, 
                     "Theorem th_4_2_9: forall A B C: Prop, A->(A->B->C)->B.", 
                     agent_parameters={'auto_limit':3})

print(res) 
print('test passed ' if res == -1 else 'failed')

process with 12692 started as
cmd:  ['opam', 'exec', '--root', '/home/pestun/dataset100', '--switch', 'ocaml-variants.4.07.1+flambda_coq-serapi.8.11.0+0.11.1', '--', 'sertop']
cwd  /home/pestun/code/pycoq/notes
-1
test passed 


In [55]:
res = await evaluate_agent(cfg, agent, 
                     "messed up input very bad239 235()*&*(^%(^PU afds ;Y a\sf\\a\sd\f asdf\\ )) should not crash but return -2", 
                     agent_parameters={'auto_limit':10})

print(res) 
print('test passed ' if res == -2 else 'failed')

process with 12695 started as
cmd:  ['opam', 'exec', '--root', '/home/pestun/dataset100', '--switch', 'ocaml-variants.4.07.1+flambda_coq-serapi.8.11.0+0.11.1', '--', 'sertop']
cwd  /home/pestun/code/pycoq/notes
notice: 0 sids in coq_stmt: messed up input very bad239 235()*&*(^%(^PU afds ;Y a\sf\a\sd asdf\ )) should not crash but return -2
-2
test passed 


## Evaluate propositions of length 4 with 2 variables and discover Peirce law

###  Init coq session for each proposition

In [73]:
# mark by * where bool_denote judgement of proposition does not agree with type inhabitation judgement 
n_args = 4
n_vars = 2
for e in db.find({'n_args':n_args, 'n_vars':n_vars}):
    th = coq_proposition(e['postfix'], n_args, n_vars, e['lex_pos'])
    res = await evaluate_agent(cfg, agent, th, agent_parameters = {'auto_limit': 10})
    print( '*' if e['bool_denote'] != (res > 0) else '', th, res, e['bool_denote'])

process with 16208 started as
cmd:  ['opam', 'exec', '--root', '/home/pestun/dataset100', '--switch', 'ocaml-variants.4.07.1+flambda_coq-serapi.8.11.0+0.11.1', '--', 'sertop', '--topfile', 'serapi_shell']
cwd  /home/pestun/code/pycoq/notes
 Theorem th_4_2_0: forall A B: Prop, (A->(A->(A->B))). -1 False
process with 16213 started as
cmd:  ['opam', 'exec', '--root', '/home/pestun/dataset100', '--switch', 'ocaml-variants.4.07.1+flambda_coq-serapi.8.11.0+0.11.1', '--', 'sertop', '--topfile', 'serapi_shell']
cwd  /home/pestun/code/pycoq/notes
 Theorem th_4_2_1: forall A B: Prop, (A->(A->(B->A))). 1 True
process with 16221 started as
cmd:  ['opam', 'exec', '--root', '/home/pestun/dataset100', '--switch', 'ocaml-variants.4.07.1+flambda_coq-serapi.8.11.0+0.11.1', '--', 'sertop', '--topfile', 'serapi_shell']
cwd  /home/pestun/code/pycoq/notes
 Theorem th_4_2_2: forall A B: Prop, (A->(A->(B->B))). 1 True
process with 16226 started as
cmd:  ['opam', 'exec', '--root', '/home/pestun/dataset100', '-

### Evaluate batch propositions by an agent in a single coq session (single core)

In [77]:
import tqdm


n_args = 4
n_vars = 2


pycoq.agent.DEBUG = False

async with pycoq.serapi.CoqSerapi(cfg, logfname="logtemp") as session:
    for e in tqdm.tqdm(db.find({'n_args':n_args, 'n_vars':n_vars})):
        prop = coq_proposition(e['postfix'], n_args, n_vars, e['lex_pos'])
        res = await evaluate_agent_in_session(session, agent, prop, agent_parameters = {'auto_limit': 10})
        print( '*' if e['bool_denote'] != (res > 0) else '', prop, res, e['bool_denote'])


0it [00:00, ?it/s]

process with 16512 started as
cmd:  ['opam', 'exec', '--root', '/home/pestun/dataset100', '--switch', 'ocaml-variants.4.07.1+flambda_coq-serapi.8.11.0+0.11.1', '--', 'sertop', '--topfile', 'serapi_shell']
cwd  /home/pestun/code/pycoq/notes


35it [00:00, 78.53it/s]

 Theorem th_4_2_0: forall A B: Prop, (A->(A->(A->B))). -1 False
 Theorem th_4_2_1: forall A B: Prop, (A->(A->(B->A))). 1 True
 Theorem th_4_2_2: forall A B: Prop, (A->(A->(B->B))). 1 True
 Theorem th_4_2_3: forall A B: Prop, (A->(B->(A->A))). 1 True
 Theorem th_4_2_4: forall A B: Prop, (A->(B->(A->B))). 1 True
 Theorem th_4_2_5: forall A B: Prop, (A->(B->(B->A))). 1 True
 Theorem th_4_2_6: forall A B: Prop, (A->(B->(B->B))). 1 True
 Theorem th_4_2_7: forall A B: Prop, (A->((A->A)->B)). -1 False
 Theorem th_4_2_8: forall A B: Prop, (A->((A->B)->A)). 1 True
 Theorem th_4_2_9: forall A B: Prop, (A->((A->B)->B)). 2 True
 Theorem th_4_2_10: forall A B: Prop, (A->((B->A)->A)). 1 True
 Theorem th_4_2_11: forall A B: Prop, (A->((B->A)->B)). -1 False
 Theorem th_4_2_12: forall A B: Prop, (A->((B->B)->A)). 1 True
 Theorem th_4_2_13: forall A B: Prop, (A->((B->B)->B)). -1 False
 Theorem th_4_2_14: forall A B: Prop, ((A->A)->(A->B)). -1 False
 Theorem th_4_2_15: forall A B: Prop, ((A->A)->(B->A)).


### Evaluate 1000 of propositions by an agent in a single coq sessions (single core)


In [78]:
# unfortunately %time, %timeit automagic does not work with async await functions yet in juypyter / ipython https://ipython.readthedocs.io/en/stable/interactive/autoawait.html
import time
async def test(n_props, auto_limit):
    t0 = time.time()
    n_args = 7
    n_vars = 3

    reslist = []
    async with pycoq.serapi.CoqSerapi(cfg, logfname="logtemp") as session:
        for e in tqdm.tqdm(db.find({'n_args':n_args, 'n_vars':n_vars})[:n_props]):
            prop = coq_proposition(e['postfix'], n_args, n_vars, e['lex_pos'])
            res = await evaluate_agent_in_session(session, agent, prop, agent_parameters = {'auto_limit': 10})
            reslist.append((n_args,n_vars,e['lex_pos'],res))

    print(f"{len(reslist) / (time.time()-t0):4.4f} proofs per seconds")

In [79]:
await test(1000, 1)
# 336.3685 proofs per seconds before stackoverflow bug fix in coq-serapi

0it [00:00, ?it/s]

process with 16539 started as
cmd:  ['opam', 'exec', '--root', '/home/pestun/dataset100', '--switch', 'ocaml-variants.4.07.1+flambda_coq-serapi.8.11.0+0.11.1', '--', 'sertop', '--topfile', 'serapi_shell']
cwd  /home/pestun/code/pycoq/notes


1000it [00:03, 316.41it/s]


311.1119 proofs per seconds


In [16]:
await test(1000, 10) 
# 175.9385 proofs per seconds before stackoverflow bug fix in coq-serapi

1000it [00:05, 198.98it/s]

196.5346 proofs per seconds


In [17]:
await test(1000, 20)
# 132.2878 proofs per seconds before stackoverflow bug fix in coq-serapi

1000it [00:04, 210.07it/s]

207.3565 proofs per seconds


In [18]:
await test(10000, 1)

10000it [01:02, 160.98it/s]


159.3971 proofs per seconds
